In [ ]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name = fn, length = len(uploaded[fn])))

Saving 식품_train.xlsx to 식품_train.xlsx
User uploaded file "식품_train.xlsx" with length 47475026 bytes


In [ ]:
import pandas as pd

df = pd.read_excel('식품_train.xlsx')
df

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,식품,46175,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,컵밥|죽,46175,1
1,2,s,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,식품,46175,a,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46175,2
2,3,c,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,식품,40543,q,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,해마로|볶음밥,40543,1
3,4,s,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,식품,40543,a,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40543,2
4,5,c,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,식품,15426,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15426,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434307,434308,s,지금 주문주신다면 내일 출고되어 토요일에 받아보실 수 있으세요. 참고 하시어 이용에...,식품,83035,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,토요일,NaN,83035,2
434308,434309,c,지금 주문했는데요 내일 꼭 받아야 합니다 가능한지요?,식품,109688,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,1
434309,434310,s,주문건은 오늘 발송됩니다. 송장번호 조회는 저녁 6시30분 이후부터 가능하세요.,식품,109688,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,2
434310,434311,c,지금 사면 수요일에 배송받을 수 있나요?,식품,135868,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,수요일,NaN,135868,1


In [ ]:
#인텐트 빈도
freq = df['인텐트 (대)'].value_counts()
print(freq)

제품          207857
배송          184069
행사           14384
포장            8704
결제            7922
교환|반품|환불      7104
웹사이트          2080
AS            1904
멤버십            288
Name: 인텐트 (대), dtype: int64


In [ ]:
#인텐트 빈도
freq2 = df['인텐트 (소)'].value_counts()
print(freq2)

질문    408424
확인     13106
요청      7342
비교      5440
Name: 인텐트 (소), dtype: int64


In [ ]:
#konlpy 설치
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.5 MB/s eta 0:00:00


# 1. 인텐트 제품 키워드 분석

In [ ]:
key1 = df[(df['인텐트 (대)']=='제품')]
key1

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
2988,2989,c,혹시 일리네스 프레소 캡슐 클래시 코랑룽고 차이가요룽고는 물양 많이 뽑아도 클래시코...,식품,109877,q,m,제품,품질,비교,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,일리네스 프레소 캡슐|클래시|코랑룽고,109877,1
2989,2990,s,룽고 제품이 클래시코 제품보다 추출량이 더 많습니다.,식품,109877,a,m,제품,품질,비교,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,룽고|클래시코,109877,2
2990,2991,c,꿀밤고구마랑 그냥 밤고구마랑 뭐가 다른 가요?,식품,123991,q,m,제품,품질,비교,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,꿀밤고구마|밤고구마,123991,1
2991,2992,s,세척 꿀밤고구마 2018년 햇꿀고구마부터 출고될 예정입니다.,식품,123991,a,m,제품,품질,비교,...,NaN,NaN,NaN,NaN,NaN,NaN,2018년,세척 꿀밤고구마|햇꿀고구마,123991,2
2992,2993,c,숄더랙을 주문하고 싶은데 냉장과 냉동의 차이가 클까요?,식품,141925,q,m,제품,품질,비교,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,숄더랙|냉장|냉동,141925,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431611,431612,s,해당 내용으로 담당 부서 전달할게요.,식품,225706,a,m,제품,불량,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225706,2
431612,431613,c,아린 맛이 있을 수도 있습니다라고 해놨지 이건 아린 맛이 아니라 독소가 너무 강해서...,식품,121096,q,m,제품,불량,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,감자,121096,1
431613,431614,s,반품택배기사 방문하면 모두 보내 주십시오. 반품 후 확인하여 조치하겠습니다.,식품,121096,a,m,제품,불량,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121096,2
431614,431615,c,13과 중 3개가 문제 잇어서 통화했어지요 기억 하시죠?,식품,14207,q,n,제품,불량,확인,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14207,1


In [ ]:
import konlpy
from konlpy.tag import Okt
from collections import Counter

발화문 = key1.loc[:,'발화문']
a = 발화문.values.tolist()

string = ' '.join(a)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter1 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter1.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('제품', 28105), ('상품', 22354), ('보관', 19750), ('유통', 15815), ('기한', 15635), ('주문', 15595), ('섭취', 13183), ('구매', 12039), ('정도', 10871), ('언제', 10657), ('건가', 9942), ('냉동', 9351), ('판매', 9271), ('문의', 9200), ('냉장', 8172), ('저희', 8009), ('경우', 7686), ('현재', 7421), ('확인', 6787), ('배송', 6396), ('발송', 6263), ('하루', 5874), ('참고', 5301), ('출고', 5174), ('사용', 4977), ('포장', 4851), ('하나요', 4646), ('개월', 4401), ('혹시', 4179), ('때문', 4146), ('차이', 3946), ('가요', 3908), ('다른', 3827), ('지금', 3737), ('양해', 3691), ('고객', 3666), ('복용', 3582), ('예정', 3534), ('품절', 3526), ('상태', 3499), ('바로', 3485), ('권장', 3466), ('고구마', 3421), ('옵션', 3415), ('오늘', 3415), ('해당', 3348), ('박스', 3152), ('추가', 3114), ('생산', 3092), ('성분', 2866), ('가격', 2838), ('선택', 2762), ('추천', 2745), ('식품', 2712), ('소스', 2707), ('이상', 2702), ('퍼센트', 2643), ('부분', 2619), ('캡슐', 2584), ('비타민', 2542), ('제조', 2504), ('김치', 2470), ('해동', 2443), ('일반', 2422), ('아이', 2419), ('조금', 2346), ('주신', 2328), ('주시', 2267), ('가루', 2259), ('모두', 2251), ('변경'

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드4.xlsx', index=False)

# 2. 인텐트 배송 키워드 분석

In [ ]:
key2 = df[(df['인텐트 (대)']=='배송')]
key2

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,식품,46175,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,컵밥|죽,46175,1
1,2,s,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,식품,46175,a,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46175,2
2,3,c,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,식품,40543,q,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,해마로|볶음밥,40543,1
3,4,s,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,식품,40543,a,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40543,2
4,5,c,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,식품,15426,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15426,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434307,434308,s,지금 주문주신다면 내일 출고되어 토요일에 받아보실 수 있으세요. 참고 하시어 이용에...,식품,83035,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,토요일,NaN,83035,2
434308,434309,c,지금 주문했는데요 내일 꼭 받아야 합니다 가능한지요?,식품,109688,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,1
434309,434310,s,주문건은 오늘 발송됩니다. 송장번호 조회는 저녁 6시30분 이후부터 가능하세요.,식품,109688,a,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109688,2
434310,434311,c,지금 사면 수요일에 배송받을 수 있나요?,식품,135868,q,m,배송,날짜,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,수요일,NaN,135868,1


In [ ]:
발화문 = key2.loc[:,'발화문']
b = 발화문.values.tolist()

string = ' '.join(b)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter2 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter2.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('배송', 74902), ('주문', 57264), ('발송', 42946), ('출고', 39362), ('오늘', 33742), ('언제', 32206), ('상품', 24868), ('택배', 18626), ('확인', 16479), ('내일', 15389), ('예정', 13181), ('수령', 11676), ('지연', 11353), ('도착', 10333), ('금일', 9578), ('현재', 9302), ('문건', 8944), ('토요일', 8875), ('당일', 8817), ('금요일', 8640), ('제품', 8298), ('월요일', 7555), ('구매', 7385), ('건가', 6738), ('오전', 6658), ('양해', 6656), ('완료', 6321), ('어제', 6295), ('지금', 6236), ('처리', 5740), ('다음', 5267), ('오후', 5152), ('순차', 5080), ('이후', 5051), ('결제', 4983), ('문의', 4634), ('저희', 4631), ('물량', 4538), ('준비', 4442), ('고객', 4339), ('이전', 4148), ('목요일', 3969), ('송장', 3883), ('폭주', 3769), ('하나요', 3739), ('해당', 3713), ('화요일', 3646), ('진행', 3565), ('정도', 3512), ('혹시', 3468), ('조회', 3446), ('마감', 3401), ('경우', 3365), ('중이', 3285), ('주시', 3165), ('수요일', 3159), ('이번', 3154), ('참고', 3094), ('출발', 3094), ('요청', 3045), ('시간', 2944), ('추석', 2932), ('포장', 2828), ('방금', 2744), ('주말', 2721), ('조금', 2641), ('번호', 2569), ('지역', 2517), ('소요', 2482), ('연휴', 2390)

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드3.xlsx', index=False)

# 3. 인텐트 포장 키워드 분석

In [ ]:
key3 = df[(df['인텐트 (대)']=='포장')]
key3

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
15886,15887,c,1번이 세일 중이어서 3번 말고 1번으로 2개 구매하려고 하는데요 이 두 개가 한 ...,식품,256707,q,m,포장,일반,질문,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,쇼핑백,256707,1
15887,15888,s,1번 1P 짜리 제품은 1P 전용 페이퍼백이 동봉되어 배송되는 제품이랍니다.,식품,256707,a,m,포장,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,페이퍼백,256707,2
15888,15889,c,포장용기가 뭔가요?,식품,38442,q,m,포장,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38442,1
15889,15890,s,네.본 제품에 사용된 포장용기의 재질은 폴리에틸렌과 폴리프로필렌 소재로 가열과정에서...,식품,38442,a,m,포장,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38442,2
15890,15891,c,택배용 박스 말고 겉박스 따로 없나요?,식품,208774,q,m,포장,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,겉박스,208774,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426693,426694,s,네 50그램은 안 되고요 냉장 보관은 10일 정도 되요.,식품,264750,a,m,포장,방식,요청,...,NaN,50그램,NaN,NaN,NaN,NaN,NaN,NaN,264750,2
426694,426695,c,상품 포장박스에 넣어서 배송해줄 수 있나요?,식품,9639,q,m,포장,방식,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9639,1
426695,426696,s,상품 개별 포장은 어려운 점 양해 부탁드립니다. 건강한 하루 보내시길 바랍니다.,식품,9639,a,m,포장,방식,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9639,2
430380,430381,c,부모님 선물로드리려 하는데 퇴근 시간이 9시 10시 이셔서요 혹시 보냉백을 좀 더 ...,식품,87062,q,m,포장,비용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,보냉백,87062,1


In [ ]:
발화문 = key3.loc[:,'발화문']
c = 발화문.values.tolist()

string = ' '.join(c)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter3 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter3.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('포장', 4739), ('박스', 2088), ('선물', 1537), ('배송', 1478), ('주문', 1325), ('상품', 1109), ('제품', 914), ('발송', 888), ('아이스팩', 881), ('구매', 785), ('아이스박스', 775), ('쇼핑', 625), ('출고', 543), ('추가', 496), ('건가', 452), ('택배', 417), ('세트', 362), ('저희', 346), ('혹시', 339), ('경우', 331), ('동봉', 312), ('상자', 299), ('냉동', 287), ('개별', 287), ('진공', 276), ('확인', 261), ('문의', 256), ('보자기', 249), ('기본', 233), ('선택', 224), ('옵션', 218), ('보관', 213), ('드라이아이스', 208), ('비닐', 200), ('현재', 199), ('하나', 199), ('상태', 193), ('양해', 190), ('요청', 185), ('일반', 181), ('별도', 179), ('참고', 176), ('종이', 172), ('판매', 168), ('날씨', 168), ('주시', 165), ('고객', 163), ('오늘', 151), ('해당', 148), ('냉장', 143), ('각각', 142), ('정도', 141), ('수량', 131), ('부분', 121), ('때문', 121), ('다른', 118), ('처리', 117), ('구입', 117), ('낱개', 116), ('묶음', 114), ('지퍼', 114), ('하나요', 112), ('가요', 108), ('그냥', 104), ('단위', 104), ('포함', 104), ('바로', 101), ('수령', 101), ('봉지', 99), ('변경', 96), ('이상', 96), ('아이스', 96), ('진행', 92), ('합포', 86), ('하루', 85), ('봉투', 82), (

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드5.xlsx', index=False)

# 4. 인텐트 결제 키워드 분석

In [ ]:
key4 = df[(df['인텐트 (대)']=='결제')]
key4

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
15672,15673,c,영수증이 필요해서 그런데요혹시 구매하고요청 드리면 제품 배송 시 영수증동봉해주실 수...,식품,104852,q,m,결제,영수증,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104852,1
15673,15674,s,택배와 함께 별도로 제공해 드리는 영수증은 없습니다. 영수증은 주문 내역에서 출력하...,식품,104852,a,m,결제,영수증,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104852,2
15674,15675,c,혹식 따로 영수증 첨부 가능할까요?,식품,164924,q,m,결제,영수증,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164924,1
15675,15676,s,간이 영수증 동봉하여 출고도와 드리겠습니다. 오늘도 즐거운 하루 되세요.,식품,164924,a,m,결제,영수증,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164924,2
15676,15677,c,현금영수증 발급 1건으로 묶어서 발급해주시면 안 되나요?,식품,199445,q,m,결제,영수증,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199445,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431511,431512,s,요청하시는 대로 시 음용 커피 함께 발송하겠습니다. 다음번 주문시에는 드시고 싶은 ...,식품,164058,a,m,결제,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,시 음용 커피|원두커피,164058,2
431512,431513,c,설전에 생다슬기 마지막 택배 발송 가능할 때 주문하고 싶은데 가능할까요?,식품,91590,q,m,결제,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,설,생다슬기,91590,1
431513,431514,s,생다슬기의 경우 최근 한파로 인해 채취가 중단되는 경우가 있다보니 정확한 날짜 안내...,식품,91590,a,m,결제,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,생다슬기,91590,2
431514,431515,c,흔히 말하는 꿀 사과 밀병든 사과는 정말 싫어하는데요청사항에 빼달라 적으면 빼주시나요?,식품,44043,q,m,결제,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,꿀 사과|사과,44043,1


In [ ]:
발화문 = key4.loc[:,'발화문']
d = 발화문.values.tolist()

string = ' '.join(d)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter4 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter4.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('주문', 3206), ('취소', 2003), ('구매', 1212), ('배송', 1127), ('언제', 872), ('상품', 806), ('발송', 766), ('확인', 574), ('처리', 562), ('출고', 547), ('결제', 509), ('판매', 475), ('금요일', 452), ('토요일', 426), ('현재', 425), ('쿠폰', 416), ('오늘', 384), ('하나요', 360), ('주시', 357), ('할인', 353), ('수령', 335), ('추가', 325), ('제품', 316), ('오전', 313), ('택배', 302), ('건가', 300), ('지금', 281), ('완료', 278), ('오후', 266), ('문의', 260), ('목요일', 254), ('저희', 251), ('양해', 235), ('적용', 232), ('요청', 229), ('문건', 220), ('월요일', 209), ('도착', 199), ('당일', 197), ('이전', 197), ('예정', 195), ('경우', 193), ('다음', 192), ('다시', 185), ('해당', 184), ('진행', 181), ('품절', 179), ('고객', 177), ('내일', 175), ('혹시', 173), ('참고', 163), ('이후', 161), ('마감', 161), ('추석', 159), ('옵션', 155), ('선택', 152), ('이상', 140), ('신청', 139), ('준비', 137), ('환불', 131), ('수량', 131), ('가격', 130), ('구입', 127), ('이번', 127), ('입금', 125), ('박스', 122), ('화요일', 119), ('변경', 119), ('수요일', 118), ('바로', 115), ('지연', 113), ('미리', 110), ('어제', 108), ('하루', 107), ('사용', 107), ('금액', 105), 

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드6.xlsx', index=False)

# 5. 인텐트 교환|반품|환불 키워드 분석

In [ ]:
key5 = df[(df['인텐트 (대)']=='교환|반품|환불')]
key5

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
47179,47180,c,지난주에 중량 미달로 문의 글 널어서 이번 주 내에 환불해준다고 하셨는데 언제 처리...,식품,80645,q,m,교환|반품|환불,시간,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80645,1
47180,47181,s,답변이 늦어진 점 양해 부탁드립니다 최대한 빠른 처리 진행토록 하겠습니다.,식품,80645,a,m,교환|반품|환불,시간,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80645,2
47181,47182,c,물품 오배송으로 교환받기로 했는데 물품은 수거해갔는데 교환 물품은 언제 오는 거죠?,식품,64468,q,m,교환|반품|환불,시간,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64468,1
47182,47183,s,제품 일시 품절로 인해 금일 발송되십니다. 내 일 받아보시게 되시고요죄송한 마음을 ...,식품,64468,a,m,교환|반품|환불,시간,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64468,2
47183,47184,c,배송 어렵다고 문자 받았었는데 환불 언제 되나요?,식품,267439,q,m,교환|반품|환불,시간,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,267439,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420131,420132,s,네 월요일 맞교환해드리겠습니다 상품 포장 부탁드립니다.,식품,125403,a,m,교환|반품|환불,시간,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125403,2
431092,431093,c,전화도 안 돼고 교 환 어려우면 차액이라도 환불해주셔야죠. 발송 전 검수 안하시는 ...,식품,138045,q,n,교환|반품|환불,비용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138045,1
431093,431094,s,크라 제측으로 맞교환요청하겠습니다.,식품,138045,a,m,교환|반품|환불,비용,요청,...,NaN,NaN,NaN,크라 제,NaN,NaN,NaN,NaN,138045,2
431094,431095,c,배송 관련해서 문제가 많은 거 같은데 그냥 환불해주세요. 반품비는 없는 거죠?,식품,78976,q,m,교환|반품|환불,비용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78976,1


In [ ]:
발화문 = key5.loc[:,'발화문']
e = 발화문.values.tolist()

string = ' '.join(e)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter5 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter5.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('반품', 2015), ('처리', 1656), ('환불', 1653), ('교환', 1569), ('상품', 1229), ('배송', 1014), ('확인', 1013), ('취소', 807), ('주문', 666), ('택배', 654), ('신청', 637), ('제품', 585), ('언제', 552), ('출고', 480), ('진행', 472), ('접수', 457), ('요청', 453), ('수거', 431), ('회수', 429), ('발송', 383), ('건가', 381), ('완료', 364), ('저희', 349), ('다시', 318), ('하나요', 311), ('문의', 296), ('오늘', 291), ('기사', 285), ('고객', 281), ('경우', 247), ('양해', 241), ('해당', 240), ('파손', 239), ('부분', 233), ('사진', 218), ('박스', 212), ('현재', 207), ('구매', 207), ('금일', 194), ('예정', 187), ('물건', 177), ('포장', 176), ('바로', 166), ('지연', 164), ('번호', 161), ('변경', 161), ('입금', 151), ('주시', 146), ('방문', 144), ('상태', 143), ('잘못', 141), ('문자', 140), ('판매', 131), ('수령', 130), ('답변', 127), ('도착', 120), ('반송', 117), ('혹시', 117), ('연락', 116), ('다른', 115), ('폐기', 114), ('전달', 112), ('문건', 112), ('승인', 112), ('송장', 111), ('내일', 105), ('그냥', 102), ('배송비', 99), ('고구마', 99), ('문제', 94), ('직접', 94), ('정도', 91), ('지금', 90), ('어제', 90), ('식품', 87), ('하나', 87), ('때문', 84)

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드7.xlsx', index=False)

# 6. 인텐트 웹사이트 키워드 분석

In [ ]:
key6 = df[(df['인텐트 (대)']=='웹사이트')]
key6

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
33617,33618,c,홈페이지 오류 아닌가요?,식품,37549,q,m,웹사이트,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37549,1
33618,33619,s,에 구구죄 송합니다 즉시 수정할게요 알려주셔서 감사합니다.,식품,37549,a,m,웹사이트,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37549,2
33619,33620,c,충북에 증평이 안 적혀 있는 이유는 왜인가요?,식품,266003,q,m,웹사이트,오류,질문,...,NaN,NaN,충북|증평,NaN,NaN,NaN,NaN,NaN,266003,1
33620,33621,s,대리점 변동으로 인해 빠져 있네요.,식품,266003,a,m,웹사이트,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,266003,2
33621,33622,c,"상품보기에는 2번 이맥콜인데 구매하기 억들어가면 2번이코카콜라로 나오네요, 무엇이 ...",식품,166973,q,m,웹사이트,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,맥콜|코카콜라,166973,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384021,384022,s,네 확인을 해본다는 게 답변이 늦어졌습니다. 교환 가능하시구요.,식품,24074,a,m,웹사이트,사용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24074,2
384022,384023,c,문의 드리고 싶은 게 있는데 전화 연결이 너무 안 되네요 전화번호 저거 말고 다른 ...,식품,72138,q,m,웹사이트,사용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72138,1
384023,384024,s,문의사항으로 문의주시면 감사하겠습니다.,식품,72138,a,m,웹사이트,사용,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72138,2
430378,430379,c,비타샵 홈피에서는 30+5 행사하던데 여기서는 안 하나요?,식품,69832,q,m,웹사이트,사용,비교,...,35.0,NaN,NaN,비타샵,NaN,NaN,NaN,NaN,69832,1


In [ ]:
발화문 = key6.loc[:,'발화문']
f = 발화문.values.tolist()

string = ' '.join(f)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter6 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter6.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('상품', 343), ('주문', 326), ('구매', 280), ('선택', 232), ('확인', 204), ('옵션', 195), ('배송', 176), ('제품', 154), ('문의', 141), ('답변', 136), ('판매', 134), ('건가', 110), ('현재', 93), ('추가', 89), ('발송', 88), ('수정', 87), ('저희', 87), ('하나요', 84), ('처리', 69), ('고객', 65), ('사진', 65), ('해당', 56), ('출고', 53), ('어디', 52), ('설명', 51), ('내용', 51), ('리뷰', 48), ('상세', 47), ('경우', 47), ('다른', 46), ('부분', 46), ('유통', 43), ('변경', 42), ('요청', 42), ('수령', 41), ('가격', 41), ('혹시', 39), ('기한', 38), ('사은', 38), ('수량', 38), ('참고', 38), ('쿠폰', 38), ('기재', 37), ('택배', 36), ('표기', 35), ('주신', 35), ('페이지', 34), ('사항', 34), ('오늘', 34), ('품절', 33), ('댓글', 33), ('직접', 33), ('가요', 32), ('여기', 31), ('번호', 31), ('다시', 30), ('양해', 30), ('후기', 30), ('완료', 29), ('말씀', 29), ('주시', 29), ('박스', 29), ('보관', 29), ('감사', 28), ('지연', 28), ('확정', 27), ('결제', 27), ('진행', 26), ('지금', 25), ('때문', 25), ('구입', 25), ('예정', 25), ('이미지', 24), ('정도', 24), ('구성', 24), ('포장', 24), ('섭취', 24), ('상태', 24), ('언제', 24), ('정보', 23), ('문자', 23), ('잘못', 23), 

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드8.xlsx', index=False)

# 7. 제품 방법 질문 & 김치 키워드 분석

In [ ]:
key7 = df[(df['인텐트 (대)']=='AS')]
key7

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
59727,59728,c,원재료를 속이 시려고 의도적으로 앞에 다가 써놓으신 거 아니시면 당장 고치세요소비자...,식품,67383,q,m,AS,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67383,1
59728,59729,s,네.담당자한테 수정해 놓으라고 전달하였습니다.,식품,67383,a,m,AS,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,67383,2
59729,59730,c,설마 같은 실수 반복하시진 않으시겠죠?,식품,78975,q,m,AS,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78975,1
59730,59731,s,유통기한 넉넉한 제품으로 발송하도록 하겠습니다.,식품,78975,a,m,AS,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78975,2
59731,59732,c,더 보내줄 수 없나요?,식품,37482,q,n,AS,일반,요청,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37482,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422223,422224,s,주문건이 많아 개별 안내드리지 못해 죄송해요.,식품,223037,a,m,AS,일반,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223037,2
422224,422225,c,후기 보니까 몇몇 사람들이 키위가 깨져서 왔다고 써 있던 데 선물해 놓고 받은 사람...,식품,17728,q,m,AS,일반,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,키위,17728,1
422225,422226,s,좋은 상품으로 선별하여 안전배송해 드리겠습니다.,식품,17728,a,m,AS,일반,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17728,2
422226,422227,c,했고 다음부터 그러지 말라고 하시고 예예하고 끊었지 않습니까?,식품,42574,q,n,AS,일반,확인,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42574,1


In [ ]:
발화문 = key7.loc[:,'발화문']
g = 발화문.values.tolist()

string = ' '.join(g)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter7 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter7.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('확인', 247), ('배송', 238), ('상품', 238), ('처리', 196), ('주문', 189), ('답변', 155), ('제품', 129), ('고객', 128), ('출고', 123), ('문의', 123), ('고구마', 113), ('판매', 104), ('취소', 104), ('저희', 101), ('반품', 100), ('건가', 93), ('발송', 90), ('구매', 80), ('택배', 77), ('사진', 66), ('환불', 66), ('양해', 63), ('현재', 61), ('해당', 61), ('전화', 60), ('오늘', 59), ('문자', 59), ('포장', 58), ('부분', 57), ('경우', 53), ('다시', 51), ('진행', 51), ('지연', 50), ('연락', 47), ('내용', 47), ('요청', 46), ('문제', 46), ('교환', 45), ('다른', 44), ('시간', 43), ('보관', 43), ('때문', 43), ('지금', 42), ('문건', 42), ('하나요', 40), ('사과', 40), ('말씀', 39), ('박스', 39), ('금일', 36), ('잘못', 35), ('누락', 35), ('불편', 34), ('센터', 34), ('먼저', 34), ('무슨', 34), ('접수', 32), ('하루', 32), ('사람', 32), ('번호', 31), ('소스', 30), ('바로', 30), ('조금', 29), ('물건', 29), ('완료', 28), ('유통', 27), ('안내', 27), ('노력', 27), ('장난', 27), ('파손', 26), ('예정', 26), ('수거', 25), ('기사', 25), ('옵션', 25), ('가격', 25), ('소비자', 24), ('기한', 24), ('주시', 24), ('사항', 24), ('신청', 24), ('전달', 23), ('실수', 23), ('폭주', 23

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드9.xlsx', index=False)

# 8. 제품 방법 질문 & 고구마 키워드 분석

In [ ]:
key8 = df[(df['인텐트 (대)']=='멤버십')]
key8

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
47155,47156,c,저기요제가 구매 후 오늘에서야 OK캐쉬백 쿠폰을 적립했는데 450점이네요 500점이...,식품,163173,q,m,멤버십,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OK캐쉬백 쿠폰,163173,1
47156,47157,s,네 캐쉬백 쿠폰 적립 시에 수수료가 붙기 때문에 그렇게 적립되는 것으로 알고 있습니다.,식품,163173,a,m,멤버십,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,163173,2
47157,47158,c,장바구니 쿠폰을 다운받아도 적용이 안 되는데 왜 그런 겅가여?,식품,5063,q,m,멤버십,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,장바구니 쿠폰,5063,1
47158,47159,s,바로 구매로 결제하지 마시고 장바구니에 담아서 장바구니로 들어가면 제품 밑에 장바구...,식품,5063,a,m,멤버십,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,장바구니 쿠폰 15퍼센트,5063,2
47159,47160,c,2번리얼7 이상품 사고픈데 왜 쿠폰이 안 보일까요?,식품,5081,q,m,멤버십,오류,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,리얼7,5081,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393769,393770,s,네 금요일에 발송 토요일에 받으실 수 있도록 보내드리겠습니다.,식품,156756,a,m,멤버십,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,금요일|토요일,NaN,156756,2
393770,393771,c,하단의 필수 입력 요청사항을 확인하고 입력해 주세요라는 곳에 뭘 입력하는 거에요?,식품,222438,q,m,멤버십,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222438,1
393771,393772,s,필수 입력 요청사항 내용 읽어보시고 하단에 내용 입력을 해주시면 되요.,식품,222438,a,m,멤버십,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,222438,2
393772,393773,c,10퍼센트 쿠폰만 보이고 20퍼센트 쿠폰은 안 보이네요 끝난 건가요?,식품,208181,q,m,멤버십,일반,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,208181,1


In [ ]:
발화문 = key8.loc[:,'발화문']
h = 발화문.values.tolist()

string = ' '.join(h)
okt = Okt()
noun = okt.nouns(string)

#형태소 빈도수
count = Counter(noun)
remove_char_counter8 = Counter({x : count[x] for x in noun if len(x) > 1})

#빈도수 높은 순서대로 정렬 (명사)
vocab_sorted = sorted(remove_char_counter8.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('쿠폰', 187), ('적용', 71), ('확인', 40), ('할인', 37), ('상품', 36), ('구매', 34), ('사용', 32), ('퍼센트', 31), ('건가', 25), ('현재', 22), ('제품', 20), ('주문', 19), ('다운', 17), ('문의', 16), ('이상', 16), ('해당', 14), ('발송', 13), ('처리', 12), ('적립', 11), ('참고', 11), ('중복', 10), ('경우', 10), ('정상', 10), ('프로', 10), ('다시', 9), ('금액', 9), ('포인트', 9), ('양해', 9), ('하나요', 9), ('완료', 9), ('어디', 9), ('종료', 9), ('행사', 9), ('저희', 8), ('언제', 8), ('다운로드', 8), ('오류', 8), ('정보', 8), ('답변', 7), ('리뷰', 7), ('고객', 7), ('이후', 7), ('소진', 7), ('지급', 7), ('하루', 7), ('입력', 7), ('캐쉬백', 6), ('결제', 6), ('사은', 6), ('추가', 6), ('오전', 6), ('예상', 6), ('일시', 6), ('내용', 6), ('발급', 6), ('판매', 6), ('진행', 6), ('장바구니', 5), ('모바일', 5), ('문자', 5), ('배송', 5), ('가격', 5), ('마일리지', 5), ('주신', 5), ('복구', 5), ('페이지', 5), ('전산', 5), ('예정', 5), ('변경', 5), ('상품권', 5), ('꿀딜', 5), ('사항', 5), ('오늘', 4), ('적립금', 4), ('취소', 4), ('상황', 4), ('선택', 4), ('이용', 4), ('설명', 4), ('상세', 4), ('번호', 4), ('커피', 4), ('이벤트', 4), ('요청', 4), ('방법', 3), ('단일', 3), ('다른', 3), ('

In [ ]:
# 데이터프레임 생성
data = pd.DataFrame(vocab_sorted, columns=['단어', '횟수'])

# 엑셀 파일로 저장
data.to_excel('인텐트 대분류별 식품 키워드10.xlsx', index=False)